In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.nn.utils import prune
from collections import OrderedDict
import os
import flwr
from flwr.common import (parameters_to_ndarrays, ndarrays_to_parameters)
import random

d:\work\BTP\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-23 12:11:35,727	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
device = torch.device("cpu")


CIFAR10

In [151]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size=10, shuffle=True)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar_testset, batch_size=10, shuffle=False)

MNIST

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=True)


In [152]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [153]:
class QuantizedNet(nn.Module):
    def __init__(self) -> None:
        super(QuantizedNet, self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.quant(x)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.dequant(x)
        return x

In [10]:
class MNISTNet(nn.Module):
    def __init__(self) -> None:
        super(MNISTNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [136]:
def set_parameters(net, parameters):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [137]:
def get_parameters(net):
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [138]:
def get_quantized_parameters(net):
    parameters = []
    
    for name,module in net.named_modules():
        if isinstance(module, torch.nn.quantized.Linear) or isinstance(module, torch.nn.quantized.Conv2d):
            parameters.append(module.weight().int_repr().cpu().numpy())
            parameters.append(module.bias().detach().cpu().numpy())
            
       
    return parameters  # Convert to NumPy array

In [5]:
def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # metrics
            epoch_loss += loss.item()
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch + 1}: loss={epoch_loss:.4f}, acc={epoch_acc:.4f}")

In [6]:
def test(net, testloader):
    """Evaluate the network on the entire test set."""
    correct, total = 0, 0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

In [12]:
model = MNISTNet()
train(model, train_loader, 5)
accuracy = test(model, test_loader)
print(f"Accuracy: {accuracy}")

Epoch 1: loss=0.0141, acc=0.9554
Epoch 2: loss=0.0059, acc=0.9819
Epoch 3: loss=0.0044, acc=0.9867
Epoch 4: loss=0.0037, acc=0.9888
Epoch 5: loss=0.0033, acc=0.9906
Accuracy: 0.9883


In [170]:
model = Net()

In [171]:
train(model, train_loader, 1)
accuracy = test(model, test_loader)

Epoch 1: loss=0.1547, acc=0.4326


In [176]:
quantized_model = torch.ao.quantization.quantize_dynamic(model, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
torch.save(quantized_model.state_dict(), "quantized_model.pth")
print(f"{os.path.getsize('quantized_model.pth')} bytes")
quantized_model

76934 bytes


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): DynamicQuantizedLinear(in_features=400, out_features=120, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (fc2): DynamicQuantizedLinear(in_features=120, out_features=84, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (fc3): DynamicQuantizedLinear(in_features=84, out_features=10, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
)

In [177]:
new = Net()
torch.ao.quantization.quantize_dynamic(new, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8,inplace=True)
new.load_state_dict(torch.load("quantized_model.pth"))
new


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): DynamicQuantizedLinear(in_features=400, out_features=120, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (fc2): DynamicQuantizedLinear(in_features=120, out_features=84, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (fc3): DynamicQuantizedLinear(in_features=84, out_features=10, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
)

In [178]:
test(new, test_loader)

0.5171

In [4]:
class NewNet(nn.Module):
    def __init__(self) -> None:
        super(NewNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        return x

In [20]:
def get_parameters(model):
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

model = NewNet()

client_best_model = NewNet()

global_model = NewNet()

parameters = get_parameters(model)
client_best_parameters = get_parameters(client_best_model)
global_best_parameters = get_parameters(global_model)

new_weights = [np.zeros_like(param) for param in parameters]

#initialize random velocities
velocities  = [np.random.randn(*param.shape) for param in parameters]


acce = 0.5
local_acce = 0.1
global_acce = 0.8


local_rand = random.random()
global_rand = random.random()


In [23]:
for index, layer in enumerate(parameters):
            new_v = acce * velocities[index]
            new_v = new_v + local_rand * (
                local_acce * (parameters[index] - layer)
            )
            new_v = new_v + global_rand * (
                global_acce * (parameters[index] - layer)
            )
            velocities[index] = new_v
            new_weights[index] = layer + new_v

In [24]:
print(velocities)

[array([[[[ 1.58110960e-01, -1.36841484e-01, -2.03821671e-01,
           1.91435746e-01, -1.18587001e-01],
         [-3.17251974e-01,  1.06113780e-01,  7.12124619e-02,
           1.23821711e-01, -2.82686346e-02],
         [-4.46684335e-01,  4.93491419e-01, -7.84243425e-02,
           2.73953568e-03,  3.85461792e-02],
         [ 4.20288735e-01,  2.94961123e-01, -3.74785165e-01,
          -9.74252692e-02, -2.47984714e-01],
         [ 1.08908471e-01,  4.82272933e-02,  4.56860668e-01,
           2.37687287e-03, -5.28230447e-02]],

        [[-3.13417795e-02,  4.22408000e-01, -1.94079121e-01,
           3.96784878e-01, -5.05784606e-02],
         [-1.33926856e-02, -1.36040024e-01, -3.99612904e-01,
           8.35785474e-02,  8.08051539e-02],
         [ 6.54409501e-01,  2.93302596e-01, -1.30874235e-01,
           4.40740694e-01, -1.69665701e-01],
         [ 1.55980588e-01,  1.31837077e-02,  3.67427699e-01,
          -1.15473251e-01, -4.18081914e-01],
         [-5.72149088e-01,  1.37654825e-01,